<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
由 <a href="https://sebastianraschka.com">Sebastian Raschka</a> 撰写的 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> 一书的补充代码<br>
<br>代码仓库：<a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# FLOPS分析
- FLOPs（每秒浮点运算次数）通过计算执行的浮点运算数量来衡量神经网络模型的计算复杂性
- 高FLOPs表示更密集的计算和能量消耗

In [ ]:
# pip install -r requirements-extra.txt

In [1]:
from importlib.metadata import version

pkgs = [
    "thop",
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

thop version: 0.1.1-2209072238
torch version: 2.5.0+cu121


&nbsp;
## 使用固定批量大小的简单基准测试

- 仅前向计算

In [5]:
import sys
sys.path.append("../01_main-chapter-code")

import torch
from thop import profile

from gpt import GPTModel


BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 2
input_tensor = torch.randint(0, 50257, (batch_size, 1024)).to(device)

for size in model_configs:
    BASE_CONFIG.update(model_configs[size])

    model = GPTModel(BASE_CONFIG).bfloat16()
    model.to(device)

    # MACS = 累乘操作
    # MACS通常被计为两个FLOPS（一个乘法和一个累加）
    macs, params = profile(model, inputs=(input_tensor,), verbose=False)
    flops = 2*macs
    print(f"{size:18}: {flops:.1e} FLOPS")

    del model
    torch.cuda.empty_cache()

gpt-small (124M)  : 5.1e+11 FLOPS
gpt-medium (355M) : 1.4e+12 FLOPS
gpt-large (774M)  : 3.2e+12 FLOPS
gpt-xl (1558M)    : 6.4e+12 FLOPS


&nbsp;
## 自动查找批量大小的简单基准测试

In [6]:
for size in model_configs:
    print(f"\nProcessing {size}")
    config = BASE_CONFIG.copy()
    config.update(model_configs[size])

    min_batch_size = 1
    max_batch_size = None
    max_possible_batch_size = 4096

    while min_batch_size <= max_possible_batch_size:
        batch_size = (min_batch_size + max_possible_batch_size) // 2
        try:
            input_tensor = torch.randint(
                0, config["vocab_size"],
                (batch_size, config["context_length"]),
                device=device
            )

            model = GPTModel(config).bfloat16().to(device)

            # MACS = multiply-accumulate operations
            # MACS are typically counted as two FLOPS (one multiply and one accumulate)
            macs, params = profile(model, inputs=(input_tensor,), verbose=False)
            flops = 2 * macs
            print(f"  Batch size {batch_size}: {flops:.1e} FLOPS")

            # If successful, try a larger batch size
            min_batch_size = batch_size + 1
            max_batch_size = batch_size

            # Clean up
            del model, input_tensor
            torch.cuda.empty_cache()

        except RuntimeError as e:
            if "out of memory" in str(e):
                # Try smaller batch size
                max_possible_batch_size = batch_size - 1

                # Clean up
                try:
                    del model, input_tensor
                    torch.cuda.empty_cache()
                except NameError:
                    pass
            else:
                raise e


Processing gpt-small (124M)
  Batch size 128: 3.2e+13 FLOPS
  Batch size 192: 4.9e+13 FLOPS
  Batch size 208: 5.3e+13 FLOPS
  Batch size 216: 5.5e+13 FLOPS
  Batch size 220: 5.6e+13 FLOPS

Processing gpt-medium (355M)
  Batch size 128: 9.3e+13 FLOPS
  Batch size 192: 1.4e+14 FLOPS

Processing gpt-large (774M)
  Batch size 128: 2.0e+14 FLOPS

Processing gpt-xl (1558M)
  Batch size 64: 2.0e+14 FLOPS
  Batch size 96: 3.1e+14 FLOPS


&nbsp;
## 自动查找批量大小和模型FLOP利用率(MFU)的基准测试

- 来自[PaLM论文](https://arxiv.org/abs/2204.02311)的模型FLOPs利用率(MFU)解释

> 提出了一种新的效率指标，该标准与实现无关，并允许更清晰的系统效率比较，称为模型FLOPs利用率（MFU）。这是指在系统以峰值FLOPs运行时，观测到的吞吐量（每秒令牌数）相对于理论最大吞吐量的比率。关键在于，“理论最大”吞吐量仅考虑计算前向+后向传递所需的操作，而不包括rematerialization


$$\text{MFU} = \frac{\text{Observed Tokens per Second}}{\text{Theoretical Max Tokens per Second}}$$

其中

$$\text{Theoretical Max Tokens per Second} = \frac{\text{Max FLOPs per Second}}{\text{Total FLOPs per Token}}$$

并且

$$\text{Tokens per Second} = \frac{\text{Batch Size} \times \text{Sequence Length}}{\text{Total Time}}$$

- 前向和后传传播

In [14]:
# GPU制造商提供的理论最大每秒浮点运算次数

flops_per_second = {
    # https://www.techpowerup.com/gpu-specs/h100-pcie-80-gb.c3899
    "H100": {
        torch.float32: 51.22e12,  # 51.22 TFLOPs for FP32 on NVIDIA H100
        torch.float16: 204.9e12,  # 204.9 TFLOPs for FP16 on NVIDIA H100
        torch.bfloat16: 204.9e12
    },
    # https://www.techpowerup.com/gpu-specs/l4.c4091
    "L4": {
        torch.float32: 30.29e12,  # 30.29 TFLOPs for FP32 on NVIDIA L4
        torch.float16: 30.29e12,  # 30.29 TFLOPs for FP16 on NVIDIA L4
        torch.bfloat16: 30.29e12
    },
    # https://www.techpowerup.com/gpu-specs/tesla-t4.c3316
    "T4": {
        torch.float32: 8.1e12,  # 8.1 TFLOPs for FP32 on NVIDIA T4
        torch.float16: 65.13e12,  # 65.13 TFLOPs for FP16 on NVIDIA T4
        torch.bfloat16: 65.13e12
    },
    # https://www.techpowerup.com/gpu-specs/a10g.c3798
    "A10G": {
        torch.float32: 31.52e12,  # 31.52 TFLOPs for FP32 on NVIDIA A10G
        torch.float16: 31.52e12,  # 31.52 TFLOPs for FP16 on NVIDIA A10G
        torch.bfloat16: 31.52e12
    },
    # https://www.techpowerup.com/gpu-specs/a100-pcie-40-gb.c3623
    "A100": {
        torch.float32: 19.49e12,  # 19.49 TFLOPs for FP32 on NVIDIA A100
        torch.float16: 77.97e12,  # 77.97 TFLOPs for FP16 on NVIDIA A100
        torch.bfloat16: 77.97e12
    },
    # https://www.techpowerup.com/gpu-specs/geforce-rtx-3080.c3621
    "RTX_3080": {
        torch.float32: 29.77e12,  # 29.77 TFLOPs for FP32 on NVIDIA RTX 3080
        torch.float16: 29.77e12,  # 29.77 TFLOPs for FP16 on NVIDIA RTX 3080
        torch.bfloat16: 29.77e12
    },
    # https://www.techpowerup.com/gpu-specs/geforce-rtx-3090.c3622
    "RTX_3090": {
        torch.float32: 35.58e12,  # 35.58 TFLOPs for FP32 on NVIDIA RTX 3090
        torch.float16: 35.58e12,  # 35.58 TFLOPs for FP16 on NVIDIA RTX 3090
        torch.bfloat16: 35.58e12
    },
    "RTX 3090 Ti": {  # 假设RTX 3090 Ti的FLOPs与RTX 3090相同
        torch.float32: 35.58e12,  # 35.58 TFLOPs for FP32 on NVIDIA RTX 3090
        torch.float16: 35.58e12,  # 35.58 TFLOPs for FP16 on NVIDIA RTX 3090
        torch.bfloat16: 35.58e12
    }
}


In [16]:
import time


def get_gpu_model(flops_per_second_dict):
    device_name = torch.cuda.get_device_name(0)  # 获取GPU设备名称
    print(device_name)
    for model in flops_per_second_dict.keys():
        if model in device_name:
            return model  # 返回匹配的GPU型号
    return "Unknown"  # 如果未找到匹配的型号，则返回默认值


gpu_model = get_gpu_model(flops_per_second)
print("GPU Moedl:", gpu_model)

if gpu_model != "Unknown":

    for size in model_configs:
        print(f"\nProcessing {size}")
        config = BASE_CONFIG.copy()
        config.update(model_configs[size])

        min_batch_size = 1
        max_batch_size = None
        max_possible_batch_size = 4096

        while min_batch_size <= max_possible_batch_size:
            batch_size = (min_batch_size + max_possible_batch_size) // 2
            try:
                input_tensor = torch.randint(
                    0, config["vocab_size"],
                    (batch_size, config["context_length"]),
                    device=device
                )

                model = GPTModel(config).bfloat16().to(device)
                model.train()

                # Start timing
                torch.cuda.synchronize()  # 同步GPU操作
                start_time = time.time()

                # Forward & backward pass
                output = model(input_tensor)  # 前向计算
                loss = output.sum()  # 计算一个虚拟损失
                loss.backward()  # 反向传播

                # End timing
                torch.cuda.synchronize()  # 同步GPU操作
                end_time = time.time()

                total_time_seconds = end_time - start_time

                # 计算前向传播的FLOPs
                macs, params = profile(model, inputs=(input_tensor,), verbose=False)
                flops_forward = 2 * macs  # 假设一个MAC等于两个FLOPs

                # 估计后向传播的FLOPs（通常为前向传播的2倍）
                flops_backward = 2 * flops_forward

                # 前向+后向传播的总FLOPs
                total_flops = flops_forward + flops_backward  # 或 total_flops = flops_forward * 3

                data_type = next(model.parameters()).dtype
                max_flops_per_second = flops_per_second[gpu_model].get(data_type, 0)

                # 计算每秒处理的token数
                tokens_processed = batch_size * config["context_length"]
                tokens_per_second = tokens_processed / total_time_seconds

                # 计算每个token的FLOPs
                flops_per_token = total_flops / tokens_processed

                # 计算理论最大每秒处理的token数
                if flops_per_token > 0:
                    theoretical_max_tokens_per_second = max_flops_per_second / flops_per_token
                else:
                    theoretical_max_tokens_per_second = 0  # 避免除以零

                # 计算MFU
                if theoretical_max_tokens_per_second > 0:
                    mfu = tokens_per_second / theoretical_max_tokens_per_second
                else:
                    mfu = 0  # 避免除以零

                print(f"  Batch size {batch_size}: Tokens/sec: {tokens_per_second:.2f}, MFU: {mfu:.4f}")

                # 如果成功，尝试更大的批量大小
                min_batch_size = batch_size + 1
                max_batch_size = batch_size

                # Clean up
                del model, input_tensor, output, loss
                torch.cuda.empty_cache()

            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    # 尝试更小的批量大小
                    max_possible_batch_size = batch_size - 1

                    # Clean up
                    try:
                        del model, input_tensor
                        torch.cuda.empty_cache()
                    except NameError:
                        pass
                else:
                    raise e

else:
    print("Unknown GPU model. Please update the flops_per_second dictionary with your GPU information.")

NVIDIA GeForce RTX 3090 Ti
GPU Moedl: RTX 3090 Ti

Processing gpt-small (124M)
  Batch size 2: Tokens/sec: 31799.26, MFU: 0.6624
  Batch size 3: Tokens/sec: 34292.25, MFU: 0.7144

Processing gpt-medium (355M)
  Batch size 8: Tokens/sec: 11475.75, MFU: 0.6840

Processing gpt-large (774M)
  Batch size 4: Tokens/sec: 4960.64, MFU: 0.6459

Processing gpt-xl (1558M)
  Batch size 2: Tokens/sec: 3603.03, MFU: 0.9448


- 值为1.0是最佳的（等于100%）
- 请注意，批量大小比之前小，因为在这里也执行反向传播，这需要更多内存